In [0]:
import pyspark
from pyspark.sql.functions import *

In [0]:
spark.conf.set(
    "fs.azure.account.key.nnrgsblobacc.dfs.core.windows.net",
    "KOC7m5T3SJnlbUb4P19ep2pbo2PIYAdadSUN9EOxSbcvp7y5OPOAR7ozRXFqnC2hJrh8WA1B+V3E+AStoZvUNQ=="
)

In [0]:
#display(dbutils.fs.ls("abfss://assesment@nnrgsblobacc.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/fact.averagecosts.dlm,fact.averagecosts.dlm,29142959,1713418287000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/fact.transactions.dlm,fact.transactions.dlm,380631122,1713418336000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.clnd.dlm,hier.clnd.dlm,276119,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.hldy.dlm,hier.hldy.dlm,545,1713418278000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.invloc.dlm,hier.invloc.dlm,2702,1713418278000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.invstatus.dlm,hier.invstatus.dlm,17004,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.possite.dlm,hier.possite.dlm,5079,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.pricestate.dlm,hier.pricestate.dlm,157,1713418279000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.prod.dlm,hier.prod.dlm,194621,1713418280000
abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.rtlloc.dlm,hier.rtlloc.dlm,3456,1713418280000


In [0]:
data = spark.read .option("header", True).option("inferSchema", True).option("delimiter", "|").csv('abfss://assesment@nnrgsblobacc.dfs.core.windows.net/hier.invloc.dlm')


In [0]:
#data.show(10)

+---+------------------+-------+-------------+
|loc|         loc_label|loctype|loctype_label|
+---+------------------+-------+-------------+
|103|  Catalog/Internet|     DC|           DC|
|119|            Retail|     DC|           DC|
|113|     The Boulevard|  Store|        Store|
|118|         Evergreen|  Store|        Store|
|125|         Arboretum|  Store|        Store|
|126|        Burr Ridge|  Store|        Store|
|127|      Park Meadows|  Store|        Store|
|128|         Southlake|  Store|        Store|
|129|Country Club Plaza|  Store|        Store|
|130|      Copley Place|  Store|        Store|
+---+------------------+-------+-------------+
only showing top 10 rows



In [0]:
# checking for null values in the columns
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).display()

loc,loc_label,loctype,loctype_label
0,0,0,0


In [0]:
#data.dtypes

[('loc', 'int'),
 ('loc_label', 'string'),
 ('loctype', 'string'),
 ('loctype_label', 'string')]

In [0]:
data = data.dropDuplicates()

In [0]:
# Boiler plate code for specific operation
# write cleaned data to the staging dat
output_container_path = 'abfss://stagingdata@nnrgsblobacc.dfs.core.windows.net/'
data.coalesce(1).write.mode('append').option('header' ,'True').format('com.databricks.spark.csv').save(output_container_path)
files = dbutils.fs.ls(output_container_path) 
output_file = [x for x in files if x.name.startswith('part-')]  # To read only part files
dbutils.fs.mv(output_file[0].path,"%s/DimLoc.csv" % output_container_path )
files_to_delete = [file.path for file in files if file.name.startswith('_')]
for file_path in files_to_delete:
    dbutils.fs.rm(file_path, True)